Importing Libraries

In [1]:
import os
import optuna
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Set the path to the directories containing your training and testing datasets
train_dir = 'E:\Python\MobileNet-V2-Batik-Image-Classification\dataset-v3[600]-5\data-train'
test_dir = 'E:\Python\MobileNet-V2-Batik-Image-Classification\dataset-v3[600]-5\data-test'

In [3]:
# Set the number of classes in your dataset
num_classes = 5

# Set the input shape for MobileNetV2
input_shape = (224, 224, 3)

# Set the number of trials for hyperparameter tuning
num_trials = 5

Preprocessing and Augmentation

In [4]:
# Create an image data generator with data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.4,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  brightness_range=[0.8, 1.2],
                                  validation_split=0.2)

#note: test data should not have to be augmented
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [5]:
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape[:2],
                                                    class_mode='categorical',
                                                    subset='training',
                                                    shuffle=True)

Found 435 images belonging to 5 classes.


In [6]:
eval_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                   target_size=input_shape[:2],
                                                   class_mode='categorical',
                                                   subset='validation',
                                                   shuffle=True)

Found 105 images belonging to 5 classes.


In [7]:
test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape[:2],
                                                  class_mode='categorical',
                                                  shuffle=False)

Found 60 images belonging to 5 classes.


Hyperparameters Tuning

In [8]:
# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameters to tune and their search spaces
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)

    # Build the MobileNetV2 model
    base_model = MobileNetV2(include_top=False, input_shape=input_shape, weights='imagenet')
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate, decay=weight_decay),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    # Train the model
    model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          validation_data=eval_generator,
          validation_steps=len(eval_generator),
          epochs=25,
          batch_size=batch_size,
          verbose=1)

    # Evaluate the model on the evaluation generator
    _, accuracy = model.evaluate(eval_generator, steps=len(eval_generator), verbose=0)

    return accuracy

In [9]:
# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=num_trials)

[I 2023-07-16 00:19:08,098] A new study created in memory with name: no-name-bfdccb6d-0b16-4e68-84a5-3389bacbb608
C:\Users\Zia\AppData\Local\Temp\ipykernel_7724\2460344492.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
C:\Users\Zia\AppData\Local\Temp\ipykernel_7724\2460344492.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)


Epoch 1/25
14/14 [==============================] - 42s 3s/step - loss: 2.7688 - accuracy: 0.5034 - val_loss: 0.6555 - val_accuracy: 0.7810
Epoch 2/25
14/14 [==============================] - 32s 2s/step - loss: 0.4836 - accuracy: 0.8000 - val_loss: 0.3417 - val_accuracy: 0.8762
Epoch 3/25
14/14 [==============================] - 32s 2s/step - loss: 0.3988 - accuracy: 0.8598 - val_loss: 0.3931 - val_accuracy: 0.8667
Epoch 4/25
14/14 [==============================] - 31s 2s/step - loss: 0.3101 - accuracy: 0.8966 - val_loss: 0.3125 - val_accuracy: 0.8857
Epoch 5/25
14/14 [==============================] - 31s 2s/step - loss: 0.1881 - accuracy: 0.9448 - val_loss: 0.2938 - val_accuracy: 0.8857
Epoch 6/25
14/14 [==============================] - 34s 2s/step - loss: 0.1467 - accuracy: 0.9517 - val_loss: 0.2340 - val_accuracy: 0.9333
Epoch 7/25
14/14 [==============================] - 35s 3s/step - loss: 0.1510 - accuracy: 0.9494 - val_loss: 0.1874 - val_accuracy: 0.9238
Epoch 8/25
14/14 [==

[I 2023-07-16 00:32:54,185] Trial 0 finished with value: 0.9142857193946838 and parameters: {'learning_rate': 0.004727242931492625, 'batch_size': 32, 'weight_decay': 1.1969071273930583e-05}. Best is trial 0 with value: 0.9142857193946838.


Epoch 1/25
14/14 [==============================] - 36s 2s/step - loss: 1.0594 - accuracy: 0.5954 - val_loss: 0.4923 - val_accuracy: 0.8286
Epoch 2/25
14/14 [==============================] - 31s 2s/step - loss: 0.4589 - accuracy: 0.8414 - val_loss: 0.3573 - val_accuracy: 0.8762
Epoch 3/25
14/14 [==============================] - 32s 2s/step - loss: 0.3720 - accuracy: 0.8552 - val_loss: 0.2779 - val_accuracy: 0.8952
Epoch 4/25
14/14 [==============================] - 32s 2s/step - loss: 0.2951 - accuracy: 0.8736 - val_loss: 0.1720 - val_accuracy: 0.9429
Epoch 5/25
14/14 [==============================] - 33s 2s/step - loss: 0.2626 - accuracy: 0.8989 - val_loss: 0.2320 - val_accuracy: 0.9048
Epoch 6/25
14/14 [==============================] - 31s 2s/step - loss: 0.1979 - accuracy: 0.9356 - val_loss: 0.2605 - val_accuracy: 0.9238
Epoch 7/25
14/14 [==============================] - 33s 2s/step - loss: 0.1721 - accuracy: 0.9563 - val_loss: 0.2309 - val_accuracy: 0.9333
Epoch 8/25
14/14 [==

In [ ]:
# Get the best hyperparameters from the study
best_params = study.best_params
best_val_accuracy = study.best_value

In [ ]:
# Rebuild the model with the best hyperparameters
best_learning_rate = best_params['learning_rate']
best_batch_size = best_params['batch_size']
best_weight_decay = best_params['weight_decay']
# best_dropout_rate = best_params['dropout_rate']

print(f"best_learning_rate : {best_learning_rate} ")
print(f"best_batch_size : {best_batch_size} ")
print(f"best_weight_decay : {best_weight_decay} ")
print(f"best_val_accuracy : {best_val_accuracy*100:.2f}%")